In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:


def get_local_alt(f):
    to_local = lambda f : f.replace('/eos/user/e/ekoenig/','/store/user/ekoenig/')
    alt_pattern = to_local(f)

    alt_glob = fc.fs.eos.glob(alt_pattern)
    if any(alt_glob):
        return alt_glob
    
    remote_glob = fc.fs.cernbox.glob(f)
    if any(remote_glob):
        alt_glob = [ to_local(f) for f in remote_glob ]
        remote_glob = [ fc.fs.cernbox.fullpath(f) for f in remote_glob ]
        fc.fs.eos.batch_copy_to(remote_glob, alt_glob)

    alt_glob = fc.fs.eos.glob(alt_pattern)
    return alt_glob

In [3]:
treekwargs = dict(
    use_gen=False,
    treename='Events',
    normalization=None,
)

f_pattern = '/eos/user/e/ekoenig/Ntuples/NanoHH4b/run2/{pairing}_2018_0L/mc/ggHH4b_tree.root'
f_sig = f_pattern.format(pairing='mindiag')
sig = ObjIter([Tree( fc.fs.cernbox.fullpath(f_sig), **treekwargs)])

# f_pattern = '/eos/user/e/ekoenig/Ntuples/NanoHH4b/run2/{pairing}_2018_0L/mc/qcd-mg_tree.root'
# f_qcd = f_pattern.format(pairing='mindiag')
# qcd = ObjIter([Tree( fc.fs.cernbox.fullpath(f_qcd), **treekwargs)])

qcd = ObjIter([])

100%|██████████| 1/1 [00:00<00:00,  1.21it/s]


In [4]:
(sig+qcd).apply(lambda t : t.reweight(t.genWeight * t.xsecWeight * t.puWeight / 1000))

[None]

In [5]:
jets = fourb.nanohh4b.get_ak4_jets(sig[0])
jets['ak4_sinphi'] = np.sin(jets.ak4_phi)
jets['ak4_cosphi'] = np.cos(jets.ak4_phi)

In [15]:
import utils.weaverUtils as weaver
model = '/eos/uscms/store/user/ekoenig/lightning/models/feynnet_lightning/nanoHH4b/training-04Oct2023/sixjet/lightning_logs/version_0'
load_feynnet = fourb.nanohh4b.f_evaluate_feynnet(model, 'onnx')
(sig + qcd).apply(load_feynnet, report=True)

f_evaluate_feynnet:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: in ListArray64 attempting to get 5, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.2/src/cpu-kernels/awkward_ListArray_getitem_jagged_apply.cpp#L43)

In [20]:
%load_ext memory_profiler

TypeError: expected str, bytes or os.PathLike object, not WeaverONNX